In [ ]:
import pandas as pd
import tempfile
import ujson as json
import re
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import csv

In [ ]:
pip install ujson

     |████████████████████████████████| 194kB 57.2MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68022 sha256=4dba1f6f8c58c97c0e6acf48ed4e76531aa54ccffd9f7324f7c4ea59b6b0e067
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# merge all

In [ ]:
reddit_data_paths_proED = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_coms.json',
                    ]

reddit_data_paths_proRec = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_coms.json',
                    ]

In [ ]:
def json_to_df(path, type):
  if ( type == "post"):
    return pd.read_json(path)[["author", "id","title", "selftext", "num_comments", "score","created_utc", "subreddit"]]
  elif ( type == "comment"):
    return pd.read_json(path)[["id", "author", "created_utc", "link_id", "parent_id", "score", "body", "subreddit"]]

In [ ]:
def merge_dataframes(reddit_data_paths):
  df_all_posts    = json_to_df(reddit_data_paths[0], "post")
  df_all_comments = json_to_df(reddit_data_paths[1], "comment")

  for i in range(2, len(reddit_data_paths), 2):
    #read
    df_posts    = json_to_df(reddit_data_paths[i], "post")
    df_comments = json_to_df(reddit_data_paths[i+1], "comment")
    #append
    df_all_posts.append(df_posts)
    df_all_comments.append(df_comments)

  #set the id as index
  posts = df_all_posts.set_index("id")
  comts = df_all_comments.set_index("id")

  #drop deleted, removed, empty bodies
  return posts[posts.selftext != "[removed]"][posts.selftext != "[deleted]"][posts.selftext != ""][posts.selftext != "None"], comts[comts.body != "[removed]"][comts.body != "[deleted]"][comts.body != ""][comts.body != "None"]

### generate deleted

In [ ]:
# it changes the df_posts given in the parameters
# like poiter...
def fill_author(df_posts):
  dic = df_posts[df_posts.author=="[deleted]"].to_dict()['author']
  df = df_posts
  for key, value in dic.items():
    s = str(tempfile.NamedTemporaryFile().name.split('/')[2][3:])
    df.loc[key, "author"] = s

### execute

In [ ]:
ed_posts,  ed_comments  = merge_dataframes(reddit_data_paths_proED)
rec_posts, rec_comments = merge_dataframes(reddit_data_paths_proRec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
fill_author(rec_posts)
fill_author(rec_comments)
fill_author(ed_posts)
fill_author(ed_comments)

In [ ]:
ed_comments

,author,created_utc,link_id,parent_id,score,body,subreddit
id,,,,,,,
crbogrx,Klairvoyant,1431837901,t3_368hc0,t3_368hc0,4,Height: 5'6\n\nCW: 125.5\n\nHW: 138\n\nLW: 78....,proED
crbop6v,Knyburg,1431838552,t3_368hc0,t3_368hc0,6,height: 5'2\nCW: 100.5\nHW: 128.3\nLW: 88\nGW:...,proED
crccsum,Klairvoyant,1431902277,t3_36aa31,t1_crccp77,1,Sorry you feel that way. You are welcomed to d...,proED
crgs5es,FascistAsparagus,1432243177,t3_36prv5,t3_36prv5,1,A lot of times what people think is fatigue fr...,proED
cry71h7,Alexa-the-hexa,1433654414,t3_38vl70,t1_cry70dl,0,My problem is mainly that I already eat not a ...,proED
cv74rby,byebyeb1rd,1442687947,t3_3lkim6,t3_3lkim6,1,**Breakfast:** Greek Yogurt-100 cal\n\n**Lunch...,proED
d6a8ueo,ifuckpineapples,1470742664,t3_4wtg6r,t3_4wtg6r,7,There were free munchkins at work and I had c&...,proED
d6a8v9c,Someone_Who_Isnt_You,1470742726,t3_4wtq5l,t1_d69zk0u,2,"Omg, we are the same size... Makes me want a l...",proED
d6a90a1,explodingeyeballs,1470743085,t3_4wtq5l,t1_d6a8v9c,2,Dude I'm the same size too!! Yeah I know right 😭,proED


In [ ]:
ed_comments[ed_comments.author == 'Panda_Melody'].body[0]

'Nothing but water! \nI was a fat cow these last 3 days and need tp make up for it. '

In [ ]:
ed_posts

,author,title,selftext,num_comments,score,created_utc,subreddit
id,,,,,,,
368gaa,Knyburg,Ice Blonde,None,2,22,1431837086,proED
368gzz,Klairvoyant,The proED Diet/Workout Plan,"\n1. No simple carbs (ie sugar, white bread)\n...",0,5,1431837639,proED
36akr0,Klairvoyant,2468 Diet,None,0,4,1431893321,proED
375qar,Knyburg,Blue Eyes,None,0,15,1432521968,proED
3gcylw,AutoModerator,"Daily Food Diary! August 09, 2015",This is a daily food diary thread for August 0...,12,2,1439136152,proED
4sbqcp,lunaticsloverspoets,Baby's First EC Stack,You guys... I don't know what a cocaine high f...,12,16,1468253829,proED
4sbsyz,HowToBeAsian25,Need some help determining goals,"I'm a very visual person (duh, I think we all ...",12,11,1468254642,proED
4sbxbp,HowToBeAsian25,How to think about food.,"IDK if this will help anyone, but it is someth...",3,25,1468255950,proED
4sc2t6,ImOkWhyDoYouAsk,[Recipe-lunch] Salad w/ Low-fat Cottage Cheese...,None,7,26,1468257610,proED


In [ ]:
ed_posts[ed_posts.author == 'THORFINN_THE_DANE'].selftext[0]

"I've run out of money and can't afford food (gonna be homeless soon). I haven't eaten all day (it's the afternoon now) and I have barely any stomach pain, I feel good. I'm sort of mentally out of it and physically weak which I love. All I wanna do is sleep forever. I want to keep this going until I slip away peacefully or am hospitalized and get to have fun IVs.\n\nI don't care much about being fat or thin though I'm slightly overweight. Would this count as an eating disorder?\n\nI'm a guy, don't know how many guys are on this forum... sorry if this was rambly."

In [ ]:
rec_comments[rec_comments.author == 'thingsarestranger']

,author,created_utc,link_id,parent_id,score,body,subreddit
id,,,,,,,
eaims40,thingsarestranger,1543256960,t3_a0f9l4,t1_eaimd4w,1,What can it do that the other one can’t? I hea...,EDAnonymous
e9rhbxw,thingsarestranger,1542307378,t3_9xdolv,t3_9xdolv,1,Same user,EDAnonymous
eb1ojru,thingsarestranger,1543893320,t3_a2wdcx,t3_a2wdcx,1,Your BMI is 20.9 not 22.,EDAnonymous
eb1p6b7,thingsarestranger,1543893865,t3_a2wdcx,t1_eb1p3gg,1,Andddd now I hate myself bc I’m fatter than I ...,EDAnonymous
e9ptb31,thingsarestranger,1542242681,t3_9x60wj,t1_e9pt9ab,1,yes!!!! It was always a good place to bounce s...,EDAnonymous
e9ptccx,thingsarestranger,1542242714,t3_9x60wj,t1_e9ptasb,1,Same here lol stopped doing it when I realized...,EDAnonymous
e9q4jp8,thingsarestranger,1542252919,t3_9x7a7i,t3_9x7a7i,1,If you can send it please :),EDAnonymous
e9q4lu9,thingsarestranger,1542252972,t3_9x79m5,t3_9x79m5,1,I would give it another day maybe he/she is go...,EDAnonymous
e9q4yob,thingsarestranger,1542253300,t3_9x6m2k,t1_e9pxzw6,1,Can I join??? Room for one more? 😅,EDAnonymous


### save

In [ ]:
'''rec_posts.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_posts_rec.csv')
rec_comments.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_comments_rec.csv')
ed_posts.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_posts_ed.csv')
ed_comments.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_comments_ed.csv')'''

#Relations

In [ ]:
def get_tx(str):
  return str[:2]

In [ ]:
def remove_tx(str):
   return str[3:]

In [ ]:
def comment_or_reply(comment):
  parent_tx = get_tx(comment["parent_id"])
  #t3_ for links
  if ( parent_tx == "t3"):
    return "comment"
  #t1_ for comments
  elif ( parent_tx == "t1" ):
    return "reply"
  return None

In [ ]:
def initialize_lists(df_posts , df_comments, post_comments, comment_replies, author_posts, author_comments):

  post_comments   = {}
  comment_replies = {}
  author_posts    = {}
  author_comments = {}

  #from posts
  for ind, row in df_posts.iterrows():
    post_comments[ind]    = []
    author_posts[row["author"]] = []

  #from comments
  for ind, row in df_comments.iterrows():
    comment_replies[ind]     = []
    author_comments[row["author"]] = []

    #if it s a reply  --> add its parent to the comments
    if (comment_or_reply(row) == "reply"):
      comment_replies[remove_tx(row["parent_id"])] = []

    #if it s a comment linked directly to a post
    #--> add the parent (post) itself to the posts list
    elif(comment_or_reply(row) == "comment"):
      post_comments[remove_tx(row["parent_id"])] = []

    post_comments[remove_tx(row["link_id"])] = []

  return post_comments, comment_replies, author_posts, author_comments

In [ ]:
#asigning to each author his things
#each post has many comments
#each author has many comments/posts
def asign_child_to_parent(df_posts, df_comments, post_comments, comment_replies, author_posts, author_comments):

  #asign post to author
  for ind, row in df_posts.iterrows():
    author_posts[row["author"]].append(ind)


  for ind, row in df_comments.iterrows():
    #asign comment to author
    author_comments[row["author"]].append(ind)

    #asign reply(comment) to comment(post)
    if (comment_or_reply(row) == "reply"):  #if it s a reply  --> add it to replies list
      if ( remove_tx(row["parent_id"]) not in comment_replies):
        comment_replies[remove_tx(row["parent_id"])] = []
      comment_replies[remove_tx(row["parent_id"])].append(ind)

    post_comments[remove_tx(row["link_id"])].append(ind)

  return post_comments, comment_replies, author_posts, author_comments

In [ ]:
def get_commenter_repliers( df_comments, author_comments, comment_replies):
  commenter_repliers = {}
  for auth_id , comments in author_comments.items():
    commenter_repliers[auth_id] = []
    for c_id in comments:
      for r_id in comment_replies[c_id]:
          commenter_repliers[auth_id].append(df_comments.loc[r_id].author)
  return commenter_repliers

In [ ]:
def get_poster_commenters( df_commenters, author_posts, post_comments):
  poster_commenters = {}
  for auth_id , posts in author_posts.items():
    poster_commenters[auth_id] = []
    for p_id in posts:
      for c_id in post_comments[p_id]:
          poster_commenters[auth_id].append(df_commenters.loc[c_id].author)
  return poster_commenters

In [ ]:
def weights_from_to(parent_childs):
  l = {}
  for key, value in parent_childs.items():
    for c in value:
      if (c, key) in l:
        l[(c, key)] += 1
      else:
        l[(c, key)] = 1
  return l

In [ ]:
def df_from_to_weights( weights , labels):
  dics = []
  for key, value in weights.items():
    l = {}
    l[labels[0]] = key[0]
    l[labels[1]]   = key[1]
    l[labels[2]]   = value
    dics.append(l)
  return pd.DataFrame(dics)

### execute

In [ ]:
def body_reply_comment(parent_childs, df_comments):
  l = []
  for key, value in parent_childs.items():

    for c in value:
      try:
        l.append((df_comments.loc[c].author, c, df_comments.loc[c].body, df_comments.loc[c].score, df_comments.loc[key].author, key, df_comments.loc[key].body, df_comments.loc[key].score))

      except KeyError:
        l.append((df_comments.loc[c].author, c, df_comments.loc[c].body, df_comments.loc[c].score, "not scrapped", key, "not scrapped", "not scrapped"))

  return pd.DataFrame(l, columns = ['replier', 'reply_id', 'reply_body', 'reply_score', 'commenter', 'comment_id', 'comment_body', 'comment_score'])

In [ ]:
def body_post_comment(parent_childs, df_posts, df_comments):
  l = []
  for key, value in parent_childs.items():

    for c in value:
      try:
        l.append((df_comments.loc[c].author, c, df_comments.loc[c].body, df_comments.loc[c].score, df_posts.loc[key].author, key, df_posts.loc[key].selftext, df_posts.loc[key].score, df_posts.loc[key].num_comments))

      except KeyError:
        l.append((df_comments.loc[c].author, c, df_comments.loc[c].body, df_comments.loc[c].score, "not scrapped", key, "not scrapped", "not scrapped", "not scrapped"))
  return pd.DataFrame(l, columns = ['commenter', 'comment_id', 'comment_body', 'comment_score', 'poster', 'poster_id', 'post_body', 'post_score', 'post_num_comments'])

In [ ]:
post_comments_rec, comment_replies_rec, author_posts_rec, author_comments_rec = initialize_lists(rec_posts , rec_comments, {}, {}, {}, {})
post_comments_rec, comment_replies_rec, author_posts_rec, author_comments_rec = asign_child_to_parent(rec_posts, rec_comments, post_comments_rec, comment_replies_rec, author_posts_rec, author_comments_rec)
poster_commenters_rec = get_poster_commenters(rec_comments, author_posts_rec, post_comments_rec)
commenter_repliers_rec = get_commenter_repliers(rec_comments, author_comments_rec, comment_replies_rec)






In [ ]:
posters = []
for key in author_posts_rec.keys():
  posters.append(key)

In [ ]:
len(posters)

3281

In [ ]:
commenters = []
for key in author_comments_rec.keys():
  commenters.append(key)

In [ ]:
len(commenters)

7490

In [ ]:
users_rec = posters + commenters

In [ ]:
len(users_rec)

10771

In [ ]:
def Remove(duplicate):
    final_list = []
    for num in duplicate:
        if num not in final_list:
            final_list.append(num)
    return final_list

In [ ]:
len(Remove(users_rec))

7853

In [ ]:
len(users_rec)

2

In [ ]:
post_comments_ed, comment_replies_ed, author_posts_ed, author_comments_ed = initialize_lists(ed_posts , ed_comments, {}, {}, {}, {})
post_comments_ed, comment_replies_ed, author_posts_ed, author_comments_ed = asign_child_to_parent(ed_posts, ed_comments, post_comments_ed, comment_replies_ed, author_posts_ed, author_comments_ed)
poster_commenters_ed = get_poster_commenters(ed_comments, author_posts_ed, post_comments_ed)
commenter_repliers_ed = get_commenter_repliers(ed_comments, author_comments_ed, comment_replies_ed)

In [ ]:
posters_ed = []
for key in author_posts_ed.keys():
  posters_ed.append(key)

In [ ]:
len(posters_ed)

4137

In [ ]:
commenters_ed = []
for key in author_comments_ed.keys():
  commenters_ed.append(key)

In [ ]:
len(commenters)

7490

In [ ]:
users_ed = posters_ed + commenters_ed

In [ ]:
len(Remove(users_ed))

11588

### save

In [ ]:
'''df_from_to_weights(weights_from_to(commenter_repliers_rec), ['replier', 'commenter', 'weight']).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_replier_commenter_weight_rec.csv')
df_from_to_weights(weights_from_to(poster_commenters_rec), ['commenter', 'poster',  'weight']).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_commenter_poster_weight_rec.csv')

df_from_to_weights(weights_from_to(commenter_repliers_ed), ['replier', 'commenter', 'weight']).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_replier_commenter_weight_ed.csv')
df_from_to_weights(weights_from_to(poster_commenters_ed), ['commenter', 'poster',  'weight']).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_commenter_poster_weight_ed.csv')'''

In [ ]:
'''body_reply_comment(comment_replies_rec, rec_comments).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_content_reply_comment_rec.csv')
body_post_comment(post_comments_rec, rec_posts, rec_comments).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_content_comment_post_rec.csv')'''


In [ ]:
'''body_reply_comment(comment_replies_ed, ed_comments).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_content_reply_comment_ed.csv')
body_post_comment(post_comments_ed, ed_posts, ed_comments).to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_content_comment_post_ed.csv')'''



1.   df_rec_posts
2.   df_rec_comments
3.   df_body_reply_comment
3.   df_body_comment_post
3.   df_weight_commenter_poster
3.   df_weight_replier_commenter

